In [1]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--seed', type=int,default=None)
parser.add_argument('--num_of_hidden_layers', type=int,default=1)
parser.add_argument('--num_of_kernels', type=int,default=64)
parser.add_argument('--dropout_rate', type=float,default=0.2)
parser.add_argument('--learning_rate', type=float,default=0.001)
parser.add_argument('--activation_function', type=str,default='relu')
parser.add_argument('--batch_size', type=int,default=1000)
parser.add_argument('--epochs', type=int,default=2)
parser.add_argument('--dataset', type=str,default='fashion_mnist')

_StoreAction(option_strings=['--dataset'], dest='dataset', nargs=None, const=None, default='fashion_mnist', type=<class 'str'>, choices=None, help=None, metavar=None)

In [2]:
"""dcs - DVC-CC-Show this cell in the py file.
args = parser.parse_args()
"""

'dcs - DVC-CC-Show this cell in the py file.\nargs = parser.parse_args()\n'

In [3]:
#dch - Hide this cell in the py file!
args = parser.parse_args('--num_of_hidden_layers 1 --num_of_kernels 8 --batch_size 100 '
                         '--epochs 2'.split())

In [4]:
import numpy as np
if not args.seed is None:
    np.random.seed(args.seed)
import tensorflow as tf
if not args.seed is None:
    tf.random.set_seed(args.seed+100)
import yaml
import time


/home/j/anaconda3/envs/dvc-cc/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/j/anaconda3/envs/dvc-cc/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/j/anaconda3/envs/dvc-cc/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/j/anaconda3/envs/dvc-cc/lib/python3.7/site-packages/ten

### LOAD DATASET


In [5]:
if args.dataset not in ['fashion_mnist','mnist','cifar10','cifar100']:
    raise ValueError('Did not find a dataset with this Name.')

num_of_tries = 0
while num_of_tries < 100:
    try:
        if args.dataset == 'fashion_mnist':
            (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
            num_of_tries = 99999999
        elif args.dataset == 'mnist':
            (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
            num_of_tries = 99999999
        elif args.dataset == 'cifar10':
            (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
            num_of_tries = 99999999
        elif args.dataset == 'cifar100':
            (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()
            num_of_tries = 99999999
        else:
            raise ValueError('Did not find a dataset with this Name.')
    except:
        if num_of_tries < 12:
            num_of_tries += 1
            time.sleep(10)
        else:
            raise ValueError('The data could not be downloaded.')

x_train, x_test = x_train / 255.0, x_test / 255.0


In [ ]:
#dch - Hide this cell in the py file!
x_train, x_test = x_train[:1000], x_test[:200] 


### BUILD MODEL


In [6]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=x_train.shape[1:]))
for i in range(args.num_of_hidden_layers):
    model.add(tf.keras.layers.Dense(args.num_of_kernels, activation=args.activation_function))
    model.add(tf.keras.layers.Dropout(args.dropout_rate))
model.add(tf.keras.layers.Dense(y_train.max()+1, activation='softmax'))

model.summary()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=args.learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

callbacks = [
  # Interrupt training if `val_loss` stops improving for over 2 epochs
  tf.keras.callbacks.EarlyStopping(patience=100, monitor='val_loss'),
  # Write TensorBoard logs to `./tensorboard` directory
  tf.keras.callbacks.TensorBoard(log_dir='tensorboard'),
  tf.keras.callbacks.ModelCheckpoint(
            filepath='model.h5',
            save_best_only=True,
            monitor='val_loss',
            verbose=2)
]


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 8)                 6280      
_________________________________________________________________
dropout (Dropout)            (None, 8)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                90        
Total params: 6,370
Trainable params: 6,370
Non-trainable params: 0
_________________________________________________________________



### TRAIN MODEL


In [7]:
model.fit(x_train, y_train, epochs=args.epochs, batch_size=args.batch_size, validation_split=0.1,
          callbacks=callbacks, verbose=2)

W0920 12:29:21.280424 139785856010048 deprecation.py:323] From /home/j/anaconda3/envs/dvc-cc/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 54000 samples, validate on 6000 samples
Epoch 1/2

Epoch 00001: val_loss improved from inf to 0.71210, saving model to model.h5
54000/54000 - 3s - loss: 1.3308 - acc: 0.5234 - val_loss: 0.7121 - val_acc: 0.7842
Epoch 2/2

Epoch 00002: val_loss improved from 0.71210 to 0.59349, saving model to model.h5
54000/54000 - 2s - loss: 0.8708 - acc: 0.6860 - val_loss: 0.5935 - val_acc: 0.8110


### TEST MODEL

model.evaluate(x_test, y_test, verbose=2)


### SAVE SUMMARY

In [8]:
summary = {'loss': float(np.min(model.history.history['loss'])),
            'val_loss': float(np.min(model.history.history['val_loss'])),
            'acc': float(np.max(model.history.history['acc'])),
            'val_acc': float(np.max(model.history.history['val_acc']))
          }
with open('summary.yml', 'w') as f:
    yaml.dump(summary, f)
